# Jupyter Notebook: [The Hidden Treasure of _delta_logֲֲֲֲ.](https://blog.devgenius.io/the-hidden-treasure-of-delta-log-482684efb36b)

In [ ]:
# 📘 Notebook: Unlocking `_delta_log` Metadata with Polars & Delta Lake

## 1. Imports & Setup
import polars as pl
from deltalake import DeltaTable
import os

delta_path = "./delta_table_example"

## 2. Initial Write
# Create and write initial DataFrame into Delta
df_initial = pl.DataFrame({
    "id": ["A", "B", "C"],
    "value": [10, 20, 30]
})
df_initial.write_delta(delta_path, mode="overwrite")

## 3. Append Additional Data
df_new = pl.DataFrame({
    "id": ["D", "E"],
    "value": [40, 50]
})
df_new.write_delta(delta_path, mode="append")

## 4. Inspect `_delta_log` Files
log_files = sorted(os.listdir(os.path.join(delta_path, "_delta_log")))
print("Log files:", log_files)

## 5. Efficient Metadata-Driven Read
# Load DeltaTable metadata
dt = DeltaTable(delta_path)

# Get version and added files metadata
versions = dt.version()  # latest version number
files = dt.files()
print("Delta version:", versions)
print("Files in latest version:", files)

## 6. Point-in-Time Table Reads
# Read specific version as Polars DataFrame
for version in [0, versions]:
    df_version = pl.read_delta(delta_path, version=version)
    print(f"\nData at version={version}:\n", df_version)

## 7. Partition Filtering via Metadata (if partitioned)
# If table is partitioned, use dt.history() or dt.schema()
schema = dt.schema()
print("Table schema:", schema)

## 8. Lazy Scan with Snapshot
ldf = pl.scan_delta(str(delta_path))
print("\nLazy scan (latest):")
print(ldf.collect())

## 9. Putting It All Together in a Pipeline
def load_version(path: str, version: int = None) -> pl.DataFrame:
    dt = DeltaTable(path)
    if version is None:
        return pl.read_delta(path)
    return pl.read_delta(path, version=version)

# Example: compare row counts across versions
df_v0 = load_version(delta_path, 0)
df_v1 = load_version(delta_path)  # latest
print("\nRow counts:", df_v0.height, df_v1.height)

## 10. Summary & Insights
print("""
✅ Utilize DeltaTable metadata to inspect `_delta_log` contents
✅ Efficiently read specific versions with minimal I/O
✅ Lazy scans reflect latest state without overhead
""")
